In [1]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [2]:
# csv 파일 로드
df = pd.read_csv("./dataset/indata_kor.csv", encoding='CP949')
df.head()

,inputs,response
0,유튜브 채널 hkcode에서는 무엇을 가르치나요?,"초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다."
1,유튜브 채널 hkcode는 누가 운영하나요?,한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다.
2,스마트금융과는 무엇을 가르치나요?,"스마트금융과는 빅데이터, 인공지능, 웹개발 및 블록체인을 가르치고 있습니다."
3,스마트금융과 등록비용은 얼마인가요?,등록비용은 국비지원 과정으로 무료 입니다.
4,스마트금융과는 1년에 몇 명을 선발하나요?,1년에 한반을 운영하고 있고 최대 27명을 선발합니다.


In [6]:
# 텍스트 분할
texts = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_text("\n".join(df.to_string()))

In [7]:
# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\human-13\AppData\Local\Temp\ipykernel_13916\920083076.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

In [9]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma2", tempeature=0.1)

C:\Users\human-13\AppData\Local\Temp\ipykernel_13916\3071692779.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma2", tempeature=0.1)


In [10]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k":1}),
    return_source_documents=True,
    verbose=False
)

In [11]:
# 채팅 함수 정의
def chat(message, history):
    # 이전 대화 기록을 ConversationalRetrievalChain 형식으로 변환
    chat_history = [(human, ai) for human, ai in history]

    # 모델을 사용하여 응답 생성
    response = qa_chain({"question": message, "chat_history": chat_history})

    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""

    return response['answer'] + source_info

In [12]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title="대학 정보 AI 챗봇",
    description="스마트금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

c:\AIProject\.venv\lib\site-packages\gradio\components\chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [ ]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

In [ ]:
demo.close()